In [ ]:
import numpy as np
import sympy
from sympy import Expr, Eq, latex, plot_implicit, tan, cot
from sympy.core import S, pi, sympify
from sympy.core.logic import fuzzy_bool
from sympy.core.numbers import Rational, oo
from sympy.core.compatibility import ordered
from sympy.core.symbol import Dummy, _uniquely_named_symbol, _symbol
from sympy.simplify import simplify, trigsimp
from sympy.functions.elementary.miscellaneous import sqrt, Max
from sympy.functions.elementary.trigonometric import cos, sin
from sympy.functions.special.elliptic_integrals import elliptic_e
from sympy.geometry.exceptions import GeometryError
from sympy.geometry.line import Ray2D, Segment2D, Line2D, LinearEntity3D
from sympy.polys import DomainError, Poly, PolynomialError
from sympy.polys.polyutils import _not_a_coeff, _nsort
from sympy.solvers import solve
from sympy.solvers.solveset import linear_coeffs
from sympy.utilities.misc import filldedent, func_name
from sympy.geometry.entity import GeometryEntity, GeometrySet
from sympy.geometry.point import Point, Point2D, Point3D
from sympy.geometry.line import Line, Segment
from sympy.geometry.util import idiff
import matplotlib.pyplot as plt
%matplotlib inline

# Занятие 15
# Алгебра
## Кривые второго порядка на плоскости: гипербола

In [ ]:
class Hyperbola(GeometrySet):
    """
     Attributes
    ==========
    center
    hradius
    vradius
    eccentricity
    periapsis
    focus_distance
    foci
    ==========
    конструктор __new__
    скопирован из Эллипса,
    изменена формула для  hradius и vradius через эсцентриситет
    """
    def __new__(
        cls, center=None, hradius=None, vradius=None, eccentricity=None, **kwargs):
        hradius = sympify(hradius)
        vradius = sympify(vradius)

        eccentricity = sympify(eccentricity)

        if center is None:
            center = Point(0, 0)
        else:
            center = Point(center, dim=2)

        if len(center) != 2:
            raise ValueError('The center of "{0}" must be a two dimensional point'.format(cls))

        if len(list(filter(lambda x: x is not None, (hradius, vradius, eccentricity)))) != 2:
            raise ValueError(filldedent('''
                Exactly two arguments of "hradius", "vradius", and
                "eccentricity" must not be None.'''))

        if eccentricity is not None:
            if hradius is None:
                hradius = vradius/sqrt( eccentricity**2 - 1)
            elif vradius is None:
                vradius = hradius*sqrt( eccentricity**2 - 1)

        if hradius == 0:
            return "Line"
        if vradius == 0:
            return "Rays"
        return GeometryEntity.__new__(cls, center, hradius, vradius, **kwargs)
    
    @property
    def ambient_dimension(self):
        return 2
    
    @property
    def center(self):
        return self.args[0]

    @property
    def hradius(self):
        return self.args[1]
    
    @property
    def vradius(self):
        return self.args[2]
    
    @property
    def focus_distance(self):
        return sqrt(self.hradius**2 + self.vradius**2)
    
    @property
    def eccentricity(self):
        """The eccentricity of the ellipse."""
        return self.focus_distance/self.hradius
    
    @property
    def periapsis(self):
        """The apoapsis of the hyperbola.
        The smallest distance between the focus and the contour."""
        return self.focus_distance-self.hradius
    @property
    def foci(self):
        return (self.center + Point(-self.focus_distance, 0), self.center + Point(self.focus_distance, 0))
    
    @property
    def major(self):
        return self.hradius
    
    @property
    def minor(self):
        return self.vradius
    
    @property
    def asymptote(self, x='x', y='y', equation=False):
        x = _symbol(x, real=True)
        y = _symbol(y, real=True)
        eq1 = Eq(y, self.center[1] + self.vradius / self.hradius * (x - self.center[0]))
        eq2 = Eq(y, self.center[1] - self.vradius / self.hradius * (x - self.center[0]))
        return [eq1, eq2]
    
    @property
    def directrix(self, x='x', equation=False):
        x = _symbol(x, real=True)
        eq1 = Eq(x, self.hradius / self.eccentricity)
        eq2 = Eq(x, - self.hradius / self.eccentricity)
        return [eq1, eq2]
        
    def equation(self, x='x', y='y', _slope=None, equation=False):
        """
        Returns the equation of a hyperbola aligned with the x and y axes;
        when slope is given, the equation returned corresponds to a hyperbola 
        with a major axis having that slope.
        Parameters
        ==========
        x : str, optional
            Label for the x-axis. Default value is 'x'.
        y : str, optional
            Label for the y-axis. Default value is 'y'.
        _slope : Expr, optional
                The slope of the major axis. Ignored when 'None'.
        Returns
        =======
        equation : sympy expression   """

        x = _symbol(x, real=True)
        y = _symbol(y, real=True)

        dx = x - self.center.x
        dy = y - self.center.y

        if _slope is not None:
            L = (dy - _slope*dx)**2
            l = (_slope*dy + dx)**2
            h = 1 + _slope**2
            a = h*self.major**2
            b = h*self.minor**2
            res= l/a - L/b 

        else:
            t1 = (dx/self.hradius)**2
            t2 = (dy/self.vradius)**2
            res = t1 - t2 
        if not equation:
            return res - 1
        else:
            return Eq(res, 1)       

# Занятие 15
# Алгебра
### Задание 1.
Построить гиперболу с центром в точке А(1, 2), полуосями 12 и 5. Вывести на экран эксцентриситет, фокусное расстояние, фокусы (с координатами), periapsis и уравнение гиперболы.

In [ ]:
Hyp1 = Hyperbola(Point(1, 2), hradius=12, vradius=5)
props = {'center': Hyp1.center, 'hradius': Hyp1.hradius, 'vradius': Hyp1.vradius,
 'eccentricity': Hyp1.eccentricity, 'periapsis': Hyp1.periapsis,
 'focus_distance': Hyp1.focus_distance, 'foci': Hyp1.foci}
for key in props:
    display(key, props[key])
    print()
display(Hyp1.equation())

'center'

Point2D(1, 2)

'hradius'

12

'vradius'

...

'focus_distance'

13

'foci'

(Point2D(-12, 2), Point2D(14, 2))

(x/12 - 1/12)**2 - (y/5 - 2/5)**2 - 1

### Задание 2.
Добавить в описание класса гипербол методы 

asymptote(self, x='x', y='y', equation=False)

directrix(self, x='x', equation=False)

Для гиперболы Задания 1 вывести на экран уравнения асимптот и директрис.

In [ ]:
Hyp1 = Hyperbola(Point(1, 2), hradius=12, vradius=5)
display(Hyp1.asymptote[0])
display(Hyp1.asymptote[1])
print()
display(Hyp1.directrix[0])
display(Hyp1.directrix[1])

Eq(y, 5*x/12 + 19/12)

Eq(y, 29/12 - 5*x/12)

Eq(x, 144/13)

Eq(x, -144/13)

### Задание 3.
Вывести на экран уравнение гиперболы из задания 1 с поворотом на 30 градусов против часовой стрелки относительно ее центра.
Изобразить повернутую гиперболу с помощью plot_implicit.

In [ ]:
eq2 = Hyp1.equation(_slope=sympy.tan(pi/6), equation=True)
display(eq2)
dict_free = {str(a): a for a in eq1.free_symbols}
x_length, y_length = -60, 60
p = plot_implicit(eq2, (dict_free['x'], x_length, y_length), (dict_free['y'], x_length, y_length),line_color='r', aspect_ratio=(1, 1), show=False, adaptive=False)
p.show()

## Уравнения директрис и асимптот при повороте на угол $\alpha$
Пусть  $a$ - горизонтальная полуось, $c$ - фокусное расстояние, $x_0$ - горизонтальная координата центра гиперболы, тогда уравнение директрис 
$$
x = x_0 \pm \frac{a^2}{c}
$$
При повороте на угол $\alpha$ получается уравнение
$$
y = y_0 -{\rm ctg} \alpha(x - x_0) \pm \frac{a^2}{c}\sqrt{1 + {\rm ctg} \alpha}
$$
Уравнение асимптоты
$$
y = y_0 \pm \frac{b}{a}(x - x_0)
$$
При повороте на угол $\alpha$ получаются уравнения
\begin{align*}
y_1 = y_0 + k_1(x - x_0), \quad 
y_2 = y_0 + k_2(x - x_0), \\ 
k_1 = \frac{b + a{\rm tg} \alpha}{a - b{\rm tg} \alpha},\quad 
k_2 = \frac{-b + a{\rm tg} \alpha}{a + b{\rm tg} \alpha}
\end{align*}
### Задание 4.
Изобразить гиперболу Задания 3 (т.е. повернутую на угол) и ее асимптоты и директрисы на одном графике plot_implicit.

In [ ]:
# Основной график
Hyp1 = Hyperbola(Point(1, 2), hradius=12, vradius=5)
eq_main = Hyp1.equation(_slope=sympy.tan(pi/6), equation=True)
dict_free = {str(a): a for a in eq2.free_symbols}
p = plot_implicit(eq_main, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='g', aspect_ratio=(1, 1), show=False, adaptive=False)

# Ассимптоты
x = _symbol("x", real=True)
y = _symbol("y", real=True)
k1 = (Hyp1.vradius + Hyp1.hradius * sympy.tan(pi/6)) / (Hyp1.hradius - Hyp1.vradius * sympy.tan(pi/6))
k2 = (-Hyp1.vradius + Hyp1.hradius * sympy.tan(pi/6)) / (Hyp1.hradius + Hyp1.vradius * sympy.tan(pi/6))
eq_asympt1 = Eq(y, Hyp1.center[1] + k1 * (x - Hyp1.center[0]))
eq_asympt2 = Eq(y, Hyp1.center[1] + k2 * (x - Hyp1.center[0]))
p.extend(plot_implicit(eq_asympt1, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='r', aspect_ratio=(1, 1), show=False, adaptive=False))
p.extend(plot_implicit(eq_asympt2, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='r', aspect_ratio=(1, 1), show=False, adaptive=False))

# ТУТ ВОПРОС
# Как получить hradius и eccentricity после поворота?
eq_direct1 = Eq(x, Hyp1.hradius / Hyp1.eccentricity)
eq_direct2 = Eq(x, -Hyp1.hradius / Hyp1.eccentricity)
p.extend(plot_implicit(eq_direct1, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='b', aspect_ratio=(1, 1), show=False, adaptive=False))
p.extend(plot_implicit(eq_direct2, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='b', aspect_ratio=(1, 1), show=False, adaptive=False))

p.show()

###  Индивидуальное задание
Определить гиперболу с центром в (-3, 4), полуосями 7 и 3.  Вывести на экран эксцентриситет, фокусное расстояние, фокусы (с координатами), periapsis, уравнение гиперболы, уравнения асимптот и директрис. 

Изобразить гиперболу, повернутую на 45 градусов против часовой стрелки относительно ее центра, соответствующие асимптоты и директрисы. Вывести на экран уравнения повернутой гиперболы, соответствующих асимптот и директрис.

In [ ]:
# Информация о гиперболе
Hyp1 = Hyperbola(Point(-3, 4), hradius=7, vradius=3)
props = {'center': Hyp1.center, 'hradius': Hyp1.hradius, 'vradius': Hyp1.vradius,
         'eccentricity': Hyp1.eccentricity, 'periapsis': Hyp1.periapsis,
         'focus_distance': Hyp1.focus_distance, 'foci': Hyp1.foci,
         'eq': Hyp1.equation(), 'assymp': Hyp1.asymptote, 'direct': Hyp1.directrix}
for key in props:
    display(key, props[key])
    print()
    
# Основной график
eq_main = Hyp1.equation(_slope=sympy.tan(pi/4), equation=True)
dict_free = {str(a): a for a in eq2.free_symbols}
p = plot_implicit(eq_main, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='g', aspect_ratio=(1, 1), show=False, adaptive=False)

# Ассимптоты
x = _symbol("x", real=True)
y = _symbol("y", real=True)
k1 = (Hyp1.vradius + Hyp1.hradius * sympy.tan(pi/4)) / (Hyp1.hradius - Hyp1.vradius * sympy.tan(pi/4))
k2 = (-Hyp1.vradius + Hyp1.hradius * sympy.tan(pi/4)) / (Hyp1.hradius + Hyp1.vradius * sympy.tan(pi/4))
eq_asympt1 = Eq(y, Hyp1.center[1] + k1 * (x - Hyp1.center[0]))
eq_asympt2 = Eq(y, Hyp1.center[1] + k2 * (x - Hyp1.center[0]))
p.extend(plot_implicit(eq_asympt1, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='r', aspect_ratio=(1, 1), show=False, adaptive=False))
p.extend(plot_implicit(eq_asympt2, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='r', aspect_ratio=(1, 1), show=False, adaptive=False))

# ТУТ ВОПРОС
# Как получить hradius и eccentricity после поворота?
eq_direct1 = Eq(x, Hyp1.hradius / Hyp1.eccentricity)
eq_direct2 = Eq(x, -Hyp1.hradius / Hyp1.eccentricity)
p.extend(plot_implicit(eq_direct1, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='b', aspect_ratio=(1, 1), show=False, adaptive=False))
p.extend(plot_implicit(eq_direct2, (dict_free['x'], -20, 20), (dict_free['y'], -20, 20), line_color='b', aspect_ratio=(1, 1), show=False, adaptive=False))

p.show()
display(eq_main, eq_asympt1, eq_asympt2, eq_direct1, eq_direct2)